In [1]:
#importing libraries
import pandas as pd
import spacy
import requests
import en_core_web_sm
import numpy as np

nlp = spacy.load("en_core_web_sm")

In [2]:
#Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Defining path to text version of novel
path= '/content/drive/MyDrive/NovelsForMLProject/FAT KEILY.txt'

In [4]:
#Diving data by paragraphs
with open(path, 'r', encoding='windows-1252') as f:
  data= [line for line in f.readlines()]


In [5]:
#Creating DataFrame
df1= pd.DataFrame(data, columns=['Paragraphs'])

In [6]:
#Cleaning dataframe
df2 = df1[df1['Paragraphs'] != '\n']
df3= df2[df2["Paragraphs"].str.contains("Chapter") == False]
df4= df3[df3["Paragraphs"].str.contains(r'[*]') == False]
df5 = df4[df4['Paragraphs'].str.len()>=10]

In [7]:
#Extracting each word from dataframe and its part-of-speech tag
def Extracting_words(df):
  features= []
  for subdf in df['Paragraphs']:
    spacy_parser= nlp(str(subdf))
    for entity in spacy_parser:
      features.append({'token': entity.text, 'pos': entity.pos_})
  return features

f= Extracting_words(df5)

In [8]:
fdf= pd.DataFrame(f)

In [9]:
#Extracting all the proper nouns, pronouns, adjectives and noun
fdf1= fdf.query("pos == 'PROPN' | pos == 'PRON' | pos == 'ADJ' | pos == 'NOUN'")
fdf2= fdf1.reset_index(drop=True)

In [10]:
#Defining lists containing keywords for hair, eyes, skin color and facial hair
hair_keywords= ['hair', 'braid', 'ponytail', 'locks','mane']
eye_keywords= ['eyes', 'orbs']
skin_keywords= ['skin']
face_hair= ['moushtache', 'beard', 'clean-shaven', 'mane']

In [11]:
NovelProtHair= pd.DataFrame(columns=['Character','Hair'])
NovelProtEyes= pd.DataFrame(columns=['Character','Eyes'])
NovelProtSkin= pd.DataFrame(columns=['Character','Skin'])
NovelProtFace= pd.DataFrame(columns=['Character','Face'])
NovelProtGender= pd.DataFrame(columns=['Character','Gender'])

In [12]:
'''For hair if hair keyword is encountered it will locate the nearest adjective and proper noun and put it in NovelProtHair dataframe
   For eyes similar patterns follows
   For skin, it will find nearest skin color (black, brown, white, etc.) and proper nouns
   Similar patterns for gender and face hair
'''

for index, ent, pp in fdf2.itertuples(index=True):
  adj=""
  if ent.lower() in hair_keywords:
    matchingindex= np.array([])
    matchingindex= np.append(matchingindex, fdf2[(fdf2['pos']=='PROPN')].index.values)
    diff= abs(matchingindex - index)
    nearest_index = matchingindex[np.argmin(diff)]
    character= fdf2.loc[nearest_index].token.split(' ')[0]
    adj= adj.join(str(fdf2[(fdf2['pos']=='ADJ') & (fdf2.index.values<= index+4) & (fdf2.index.values>=index-4)].token.values))
    NovelProtHair.loc[len(NovelProtHair.index)] = [character, adj]
  if ent.lower() in eye_keywords:
    matchingindex= np.array([])
    matchingindex= np.append(matchingindex, fdf2[(fdf2['pos']=='PROPN')].index.values)
    diff= abs(matchingindex - index)
    nearest_index = matchingindex[np.argmin(diff)]
    character= fdf2.loc[nearest_index].token.split(' ')[0]
    adj= adj.join(str(fdf2[(fdf2['pos']=='ADJ') & (fdf2.index.values<= index+4) & (fdf2.index.values>=index-4)].token.values))
    NovelProtEyes.loc[len(NovelProtEyes.index)] = [character, adj]
  if ent.lower() in skin_keywords:
    matchingindex= np.array([])
    matchingindex= np.append(matchingindex, fdf2[(fdf2['pos']=='PROPN')].index.values)
    diff= abs(matchingindex - index)
    nearest_index = matchingindex[np.argmin(diff)]
    character= fdf2.loc[nearest_index].token.split(' ')[0]
    adj= adj.join(str(fdf2[((fdf2['token']=='white') | (fdf2['token']=='black') | (fdf2['token']=='brown') | (fdf2['token']=='fair')) & (fdf2.index.values<= index+4) & (fdf2.index.values>=index-4)].token.values))
    NovelProtSkin.loc[len(NovelProtSkin.index)] = [character, adj]
  if pp=='PROPN':
    matchingindex= np.array([])
    matchingindex= np.append(matchingindex, fdf2[(fdf2['pos']=='PRON') & (fdf2.index.values<= index+10) & (fdf2.index.values>=index-10)].index.values)
    diff= abs(matchingindex - index)
    if len(matchingindex)==0:
      continue
    else:
      nearest_index = matchingindex[np.argmin(diff)]
      gender= ""
      if (fdf2.loc[nearest_index].token.lower() == 'he' or fdf2.loc[nearest_index].token.lower()=='she'):
        gender= fdf2.loc[nearest_index].token.lower()
        NovelProtGender.loc[len(NovelProtGender.index)] = [ent, gender]
  if ent.lower() in face_hair:
    matchingindex= np.array([])
    matchingindex= np.append(matchingindex, fdf2[(fdf2['pos']=='PROPN')].index.values)
    diff= abs(matchingindex - index)
    nearest_index = matchingindex[np.argmin(diff)]
    character= fdf2.loc[nearest_index].token.split(' ')[0]
    NovelProtFace.loc[len(NovelProtFace.index)] = [character, ent]

In [13]:
#Filtering out character which are not frequently repeated
NovelProtHair1= NovelProtHair.groupby('Character').filter(lambda x : len(x)>1)
NovelProtEyes1= NovelProtEyes.groupby('Character').filter(lambda x : len(x)>5)
NovelProtSkin1= NovelProtSkin.groupby('Character').filter(lambda x : len(x)>=1)
NovelProtGender1= NovelProtGender.groupby('Character').filter(lambda x : len(x)>=10)
NovelProtFace1= NovelProtFace.groupby('Character').filter(lambda x : len(x)>=1)

In [14]:
NovelProtHair2= NovelProtHair1
NovelProtEyes2= NovelProtEyes1
NovelProtSkin2= NovelProtSkin1
NovelProtGender2= NovelProtGender1
NovelProtFace2= NovelProtFace1

In [15]:
#Joining all rows with similar character names
NovelProtHair2= NovelProtHair1.groupby('Character')['Hair'].apply(' '.join).reset_index()
NovelProtEyes2= NovelProtEyes1.groupby('Character')['Eyes'].apply(' '.join).reset_index()
NovelProtSkin2= NovelProtSkin1.groupby('Character')['Skin'].apply(' '.join).reset_index()
NovelProtFace2= NovelProtFace1.groupby('Character', group_keys=False)['Face'].apply(' '.join).reset_index()
NovelProtGender2= NovelProtGender1.groupby('Character')['Gender'].apply(' '.join).reset_index()

In [16]:
#Removing '[',',']' from strings
NovelProtHair2.Hair= NovelProtHair2.Hair.str.replace('[','').str.replace(']','').str.replace('\'','')
NovelProtEyes2.Eyes= NovelProtEyes2.Eyes.str.replace('[','').str.replace(']','').str.replace('\'','')
NovelProtSkin2.Skin= NovelProtSkin2.Skin.str.replace('[','').str.replace(']','').str.replace('\'','')
NovelProtFace2.Face= NovelProtFace2.Face.str.replace('[','').str.replace(']','').str.replace('\'','')

<ipython-input-16-febb76bfff53>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NovelProtHair2.Hair= NovelProtHair2.Hair.str.replace('[','').str.replace(']','').str.replace('\'','')
<ipython-input-16-febb76bfff53>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NovelProtEyes2.Eyes= NovelProtEyes2.Eyes.str.replace('[','').str.replace(']','').str.replace('\'','')
<ipython-input-16-febb76bfff53>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NovelProtSkin2.Skin= NovelProtSkin2.Skin.str.replace('[','').str.replace(']',''

In [17]:
from collections import Counter

#To collect top two most frequent words, here adjectives
def get_top_two_frequent_words(df, column_name):
    top_two_words = []
    for row in df[column_name]:
        if pd.isna(row):  # Skip empty rows
            top_two_words.append([])
            continue

        words = row.split()  # Split the row into individual words
        word_counts = Counter(words)  # Count the frequency of each word
        most_common_words = word_counts.most_common(2)  # Get the top two most frequent words
        top_two_words.append([word[0] for word in most_common_words])  # Append the top two words to the list

    df['TopTwoWords'] = top_two_words  # Create a new column with the top two words
    return df

#To collect top frequent word from all adjectives
def get_top_frequent_word(df, column_name):
    top_word = []
    for row in df[column_name]:
        if pd.isna(row):  # Skip empty rows
            top_word.append([])
            continue

        words = row.split()  # Split the row into individual words
        word_counts = Counter(words)  # Count the frequency of each word
        most_common_word = word_counts.most_common(1)  # Get the top most frequent word
        top_word.append([word[0] for word in most_common_word])  # Append the top word to the list

    df['TopWord'] = top_word  # Create a new column with the top word
    return df

In [18]:
NovelProtHair3= get_top_two_frequent_words(NovelProtHair2, 'Hair')
NovelProtEyes3= get_top_two_frequent_words(NovelProtEyes2, 'Eyes')
NovelProtSkin3= get_top_frequent_word(NovelProtSkin2,'Skin')
NovelProtFace3= get_top_frequent_word(NovelProtFace2,'Face')
NovelProtGender3= get_top_frequent_word(NovelProtGender2,'Gender')

In [19]:
NovelProtHair4= NovelProtHair3.drop(['Hair'],axis='columns').rename(columns={'TopTwoWords':'Hair'})
NovelProtEyes4= NovelProtEyes3.drop(['Eyes'],axis='columns').rename(columns={'TopTwoWords':'Eyes'})
NovelProtSkin4= NovelProtSkin3.drop(['Skin'],axis='columns').rename(columns={'TopWord':'Skin'})
NovelProtFace4= NovelProtFace3.drop(['Face'],axis='columns').rename(columns={'TopWord':'Face'})
NovelProtGender4= NovelProtGender3.drop(['Gender'],axis='columns').rename(columns={'TopWord':'Gender'})

In [20]:
#Merging Dataframes
NovelProt1= pd.merge(NovelProtHair4, NovelProtEyes4, on='Character', how='outer')
NovelProt2= pd.merge(NovelProt1, NovelProtSkin4, on='Character', how='outer')
NovelProt3= pd.merge(NovelProt2, NovelProtFace4, on='Character', how='outer')
NovelProt4= pd.merge(NovelProt3, NovelProtGender4, on='Character', how='outer')

In [21]:
#dropping rows having more than 2 NaN values in their columns
threshold = 2
Novel_filtered = NovelProt4.dropna(thresh=NovelProt4.shape[1] - threshold + 1)

In [22]:
#Creating dataframe that contains prompt for stable diffusion
descriptions= pd.DataFrame(columns=['Character','des'])
for index, row in Novel_filtered.iterrows():
    character = row['Character']
    hair = ' '.join(row['Hair'])
    eyes = ' '.join(row['Eyes'])
    skin = ' '.join(row['Skin'])
    gender = ' '.join(row['Gender'])

    if gender == 'she':
        gender = 'woman'
    elif gender == 'he':
        gender = 'man'
    descriptions.loc[len(descriptions.index)] = [character, f"{gender} with {hair} hair, {eyes} eyes, {skin} skin."]


In [23]:
descriptions.head()

,Character,des
0,Addison,"woman with curly brown hair, afraid glaring ey..."
1,James,"man with wet brown hair, dark furious eyes, fa..."
2,Lucas,"man with pinkish wild hair, other green eyes, ..."


In [24]:
descriptions.to_csv("/content/drive/MyDrive/'descriptions.csv'", index=False)